In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import utilities
import os

In [2]:
datasets_binary = ['BeetleFly','BirdChicken','Coffee','Computers','DistalPhalanxOutlineCorrect','Earthquakes','ECG200',
                   'ECGFiveDays','FordA','FordB','GunPoint','Ham','HandOutlines','Herring','ItalyPowerDemand','Lightning2',
                   'MiddlePhalanxOutlineCorrect', 'MoteStrain','PhalangesOutlinesCorrect','ProximalPhalanxOutlineCorrect',
                   'ShapeletSim','SonyAIBORobotSurface1','SonyAIBORobotSurface2','Strawberry','ToeSegmentation1','ToeSegmentation2',
                   'TwoLeadECG','Wafer','Wine','WormsTwoClass','Yoga','Chinatown','DodgerLoopGame','DodgerLoopWeekend',
                   'FreezerRegularTrain','FreezerSmallTrain','GunPointAgeSpan','GunPointMaleVersusFemale','GunPointOldVersusYoung',
                   'HouseTwenty','PowerCons','SemgHandGenderCh2']
datasets_small = ['BeetleFly', 'BirdChicken', 'Coffee', 'Computers', 'DistalPhalanxOutlineCorrect', 'Earthquakes', 'ECG200', 
                  'ECGFiveDays', 'GunPoint', 'Ham', 'Herring', 'Lightning2', 'MiddlePhalanxOutlineCorrect', 'ProximalPhalanxOutlineCorrect', 
                  'ShapeletSim', 'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2', 'Strawberry', 'ToeSegmentation1', 'ToeSegmentation2', 'Wine', 
                  'WormsTwoClass', 'Chinatown', 'DodgerLoopGame', 'DodgerLoopWeekend', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 
                  'GunPointOldVersusYoung', 'HouseTwenty', 'PowerCons', 'SemgHandGenderCh2']
dataset_large=['FordA', 'FordB', 'HandOutlines', 'ItalyPowerDemand', 'MoteStrain', 'PhalangesOutlinesCorrect', 
               'TwoLeadECG', 'Wafer', 'Yoga', 'FreezerRegularTrain', 'FreezerSmallTrain']


### Implement shapelet distance

In [3]:
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import itertools
from itertools import permutations
from itertools import repeat
from scipy.spatial import distance
import pandas as pd
import time

In [4]:
def min_distance(S,T):
    
    if S.all()==None or T.all()==None:
        return np.inf
    assert isinstance(S,np.ndarray)
    assert isinstance(T,np.ndarray) 
    assert isinstance(S[0],float)
    assert isinstance(T[0],float)
    
    if len(S)>len(T):
        aux_S=T
        T=S
        S=aux_S
    
    dist_list=[]
    m=len(T)
    w=len(S)
       
    for i in range(m-w+1):
        dist=np.linalg.norm(T[i:i+w]-S)
        dist_list.append(dist)
    
    return min(dist_list)

In [5]:
#K(T1,T2)=K_s(vectorise(T1,S), VECTORIZE(T2,S))=(v1-v2)^2
def min_sub_distance(T1,T2,k=None):
    #T1,T2 should be lists of subsequences
    S1=subsequences1d(T1,k)
    S2=subsequences1d(T2,k)
    assert isinstance(S1,np.ndarray)
    assert isinstance(S2,np.ndarray) 
    assert isinstance(S1[0],np.ndarray)
    assert isinstance(S2[0],np.ndarray)
    assert isinstance(S1[0][0],float)
    assert isinstance(S2[0][0],float)
    
    #union
    S=np.unique(np.concatenate((S1,S2), axis=0),axis=0)
    
   
    v1=dist_vectorize(T1,S)
    v2=dist_vectorize(T2,S)
    
    return np.linalg.norm(v1-v2, ord=2)  

def subsequences2d(T, k=None): 
    assert isinstance(T,np.ndarray)
    assert isinstance(T[0],np.ndarray)
    assert isinstance(T[0][0],float)
    
    m,n = T.shape
    if k==None:
        k=int(np.log(n+1))+1
    # INPUTS :
    # a is array
    # L is length of array along axis=1 to be cut for forming each subarray

    # Length of 3D output array along its axis=1
    #print(T.shape)
    nd0 = T.shape[1] - k + 1

    # Store shape and strides info
    s0,s1 = T.strides
    
    
    # Finally use strides to get the 3D array view
    return np.lib.stride_tricks.as_strided(T, shape=(m,nd0,k), strides=(s0,s1,s1))

def subsequences1d(arr, m=None):
    assert isinstance(arr,np.ndarray)
    assert isinstance(arr[0],float)
    
    if m==None:
        m=int(np.log(arr.shape[0]+1))+1
    n = arr.shape[0] - m + 1
    s = arr.itemsize
    #print(m,arr.shape[0])
    return np.lib.stride_tricks.as_strided(arr, shape=(n,m), strides=(s,s))    
    

### minimum shapelet distance

##### slower methods for intersection

In [48]:
def eculidean(s1,s2):
    return np.linalg.norm(s1-s2)

def early_abandon(S1,S2,epsilon):
    s1_remove=[]
    for i in range(0,len(S1)):
        dists=list(map(eculidean,S2,itertools.repeat(S1[i],len(S2))))
        #print("for ",S1[i],S2,dists)
        s2_remove=[j for j,v in enumerate(dists) if v < epsilon]
        
        #print("remove: ", s2_remove)
        if len(s2_remove)>0:
            s1_remove.append(i)
            S2=np.delete(S2,s2_remove,0)
    S1=np.delete(S1,s1_remove,0)
    #print("S1,S2 in early abandon: ", S1,S2)
    return S1,S2
def split_by_step(length,step):
    n=int(length/step)
    cut_point=[0]
    pairs=[]
    for i in range(1,n+1):
        cut_point.append(i*step)
    cut_point.append(length-1)
    for i in range(0,len(cut_point)-1):
        pairs.append(cut_point[i:i+2])
    return pairs
def get_dist_matrix(A,B,epsilon):
    s_len=A.shape[1]
    A_len=A.shape[0]
    B_len=B.shape[0]
    splitsA=[[0,A_len]]
    splitsB=[[0,B_len]]
    if (A_len>2):
        splitsA=split_by_step(A_len,2)
    if(B_len>2):
        splitsB=split_by_step(B_len,2)
    index1=[]
    index2=[]
    shapelet=[]
    for i in splitsA:
        for j in splitsB:
            Ai=A[i[0]:i[1],:]
            Bj=B[j[0]:j[1],:]
            print("A and B:")
            print(Ai)
            print(Bj)
            dist_mat=distance.cdist(Ai,Bj, 'euclidean') 
            index=np.argwhere(dist_mat<=epsilon)
            index=index.T
            #print('original',index)
            index[0]=index[0]+i[0]
            
            index1+=index[0].tolist()
            index[1]=index[1]+j[0]
            index2+=index[1].tolist()
            #print('after add',index[0],index[1],index)
            #UNION-INTERSECTION
    index1=np.unique(np.array(index[0]).flatten())
    index2=np.unique(np.array(index[1]).flatten())
    full_index1=np.arange(0,A.shape[0])
    full_index2=np.arange(0,B.shape[0])
   
    shapelet1=A[list(set(full_index1)-set(index1))]
    shapelet2=B[list(set(full_index2)-set(index2))]

    shapelet.append(np.concatenate((shapelet1,shapelet2), axis=0))
    return shapelet

In [82]:
def get_dynamic_dist_matrix(A,B,epsilon):
    
    s_len=A.shape[1]
    A_len=A.shape[0]
    B_len=B.shape[0]

    index1=[]
    index2=[]
    shapelet=[]
    while(A.shape[0]>20000 and B.shape[0]>20000):
        
        Ai=A[0:20000,:]
        Bj=B[0:20000,:]
        
        dist_mat=distance.cdist(Ai,Bj, 'euclidean') 
        index=np.argwhere(dist_mat<=epsilon)
        index=index.T
        #print('original',index)
        A=np.delete(A,index[0],axis=0)
        B=np.delete(B,index[1],axis=0)
   
    dist_mat=distance.cdist(A,B, 'euclidean') 
    index=np.argwhere(dist_mat<=epsilon)
    index=index.T
    A=np.delete(A,index[0],axis=0)
    B=np.delete(B,index[1],axis=0) 
    #print(A,B,type(A))
    return np.concatenate((A,B), axis=0)

In [ ]:
import numpy as np
def intersection_view(A,B):
    nrows, ncols = A.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)],
           'formats':ncols * [A.dtype]}
    indexA=np.nonzero(np.in1d(A.view('i,i').reshape(-1), B.view('i,i').reshape(-1)))[0]
    indexB=np.nonzero(np.in1d(B.view('i,i').reshape(-1), A.view('i,i').reshape(-1)))[0]
    return indexA,indexB


#### scipy.distance.cdist for intersection

In [83]:

def separate_dataset(train,test):
    labels=np.unique(test)
    X1=train[test==labels[0]]
    X2=train[test==labels[1]]
    return X1,X2

def intersection(X,Y,m=None,epsilon=None):
    
    if m==None:
        m=int(np.log(X.shape[1]+1))+1
    if epsilon==None:
        epsilon=(np.amax(X)-np.amin(X))*0.01*m
    
    #split dataset
    X1,X2=separate_dataset(X,Y)
    
    #extract subsequences
    S1=subsequences2d(X1)
    S2=subsequences2d(X2)
    S1=np.unique(S1.reshape(S1.shape[0]*S1.shape[1],S1.shape[2]),axis=0)
    S2=np.unique(S2.reshape(S2.shape[0]*S2.shape[1],S2.shape[2]),axis=0)
    
    print('sub size: ',S1.shape,S2.shape)
    shapelets=get_dynamic_dist_matrix(S1,S2,epsilon)
   
    return shapelets

def intersect_2d(A,B):
    return np.array([x for x in set(tuple(x) for x in A) & set(tuple(x) for x in B)])

def dist_vectorize(T,shapelets):
    # check shapelets is a 2d array
    assert isinstance(shapelets,np.ndarray)
    assert isinstance(shapelets[0],np.ndarray)
    assert isinstance(shapelets[0][0],float)
    
    # check T is a 1d array
    assert isinstance(T,np.ndarray)
    assert isinstance(T[0], float)
    
    dist_list=[]
    m=len(T)
    w=len(shapelets[0])
    subsequences=subsequences1d(T,w) 
    #print(subsequences)
    dist_mat=distance.cdist(shapelets,subsequences)
    
    return dist_mat.min(axis=1)
    #calculate the s-T distance vector
    #u=map(min_distance,shapelets,repeat(T))
    #return np.array(list(u))

def min_shapelet_distance(T1,T2,shapelets,k=None):
    
    assert isinstance(T1,np.ndarray)
    assert isinstance(T2,np.ndarray) 
    assert isinstance(T1[0],float)
    assert isinstance(T2[0],float)
    assert isinstance(shapelets,np.ndarray)
    assert isinstance(shapelets[0],np.ndarray)
    assert isinstance(shapelets[0][0],float)
    
    #calculate the distance vector where each element is min_dist between shapelet and T
    start=time.time()
    V1=dist_vectorize(T1,shapelets)
    V2=dist_vectorize(T2,shapelets)
    #print('time vectorize: ', time.time()-start)
    
    return np.linalg.norm(V1-V2, ord=2)    

### train and test datasets

In [ ]:
#train and test all 42 datasets
import time
dataset_list=[]
auroc_list=[]
auprc_list=[]
#['MoteStrain','TwoLeadECG','HouseTwenty']
for dataset in ['Strawberry','WormsTwoClass','SemgHandGenderCh2','MoteStrain','TwoLeadECG','HouseTwenty']:
    start_time=time.time()
    X_train, y_train, X_test, y_test = utilities.get_ucr_dataset('../UCRArchive_2018/',dataset)
    # calculate shapelets
    shapelets=intersection(X_train,y_train)
    
    #KNN
    clf = KNeighborsClassifier(n_neighbors=1,metric=min_shapelet_distance,metric_params={'shapelets':shapelets})
    clf.fit(X_train, y_train)
    print("--- fit in %s seconds ---" % (time.time() - start_time))
    y_pred = clf.predict_proba(X_test)
    auroc=roc_auc_score(y_test, y_pred[:, 1])
    auprc=average_precision_score(y_test, y_pred[:,1])
    print(dataset, " AUROC is: ", auroc," AUPRC is: ", auprc)
    print("--- %s seconds ---" % (time.time() - start_time))
    f=open('KNN_min_shapelet_dist2.csv','a')
    np.savetxt(f, np.array([dataset, auroc,auprc]).reshape(1,3), delimiter=',',fmt="%s")  
    f.write("\n")
    f.close()       
    #37

sub size:  (50370, 6) (90620, 6)
--- fit in 121.08548521995544 seconds ---


In [17]:
np.savetxt('sub_min_dist.csv', [p for p in zip(dataset_list, auroc_list,auprc_list)], delimiter=',',fmt="%s")

In [13]:
#Grid Search over datasets, subsequence length as a hyper-parameter 
dataset_list=[]
auroc_list=[]
auprc_list=[]
param_list=[]
parameters = {'n_neighbors':[1],'metric_params':[{'k':1},{'k':2},{'k':3},{'k':4}]}

for dataset in datasets_small[:1]:
    X_train, y_train, X_test, y_test = utilities.get_ucr_dataset('../UCRArchive_2018/',dataset)
    clf = GridSearchCV(KNeighborsClassifier(metric=pairwise_min_shapelet),parameters, cv=5, verbose=1)
    clf.fit(X_train, y_train)
    
    #get best estimator
    print(clf.best_params_)
    opt_clf=clf.best_estimator_
    
    y_pred = opt_clf.predict_proba(X_test)
    
    auroc=roc_auc_score(y_test, y_pred[:, 1])
    auprc=average_precision_score(y_test, y_pred[:,1])
    print(dataset, " AUROC is: ", auroc," AUPRC is: ", auprc)
    dataset_list.append(dataset)
    auroc_list.append(auroc)
    auprc_list.append(auprc)
    param_list.append(clf.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
1 512
1 512


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512
1 512


KeyboardInterrupt: 

In [ ]:
np.savetxt('pairwise_min_shapelet_KNN.csv', [p for p in zip(dataset_list, auroc_list,auprc_list,param_list)], delimiter=',',fmt="%s")

In [43]:
V1=np.array([0,0,1,2])
V2=np.array([3,4,0,0])
v3=np.array([1,2,0,0])
v4=np.array([0,0,3,4])
a=np.linalg.norm(V1-V2, ord=2)
b=np.linalg.norm(v3-v4, ord=2)
print(a==b)

True
